In [8]:
"""
create a test for create_memorization_sample_across.py

Need to 
1. upload json files (in the format of the pre-training data) to 2 s3 buckets for args.train_files and args.val_files
2. have multiple train and val files so we can test aggregation
3. have samples in the val files that meet and don't meet the freqeuncy treshold overall, length threshold

"""

import json
import os
import s3_accessor

content_column = "text"
filter_columns = ["identity_attack", "insult","obscene","severe_toxicity","sexual_explicit","threat","toxicity"]

def package_and_upload(text_data, file_name, upload_bucket):
    if os.path.isfile(file_name):
        print(f'removing {file_name}')
        os.remove(file_name)
    with open(file_name, 'a') as f:
        for text in text_data:
            json_v = {'text': text, 'toxicity': '0'}
            f.write(json.dumps(json_v))
            f.write('\n')
            
    s3_accessor.upload(f"{upload_bucket}{file_name}", file_name)
            

unique_lines = [
    "Offham Hill is a causewayed enclosure near Lewes, East Sussex, England. Causewayed enclosures were built in England from shortly before 3700 BC until about 3300 BC; they are characterized by the full or partial enclosure of an area with ditches that are interrupted by gaps, or causeways. Their purpose is not known; they may have been settlements, meeting places, or ritual sites. The site was identified as a possible causewayed enclosure in 1964 by a member of the Sussex Archaeological Society. The Ordnance Survey inspected the site in 1972 and recommended an excavation, which was carried out in 1976 by Peter Drewett.",
    "Kiyohara Yukinobu (1643–1682) was a Japanese painter and one of the foremost women identified with the Kanō school. Her father Kusumi Morikage was also a painter and her mother Kuniko was the niece of his longtime teacher and patron Kanō Tan'yū. Yukinobu lived in Kyoto and likely studied under her father. Her work covered a wide variety of formats ranging from small scrolls to large screens.",
    "A constitutional referendum was held in Tajikistan on 26 September 1999.[1] The changes included legalising religious political parties, introducing a bicameral parliament, and lengthening the president's term from five to seven years. They were approved by 75% of voters, with a turnout of 93%.[2]",
    "Sokol District (Russian: райо́н Со́кол) is an administrative district (raion), one of the sixteen in Northern Administrative Okrug of the federal city of Moscow, Russia.[1] The area of the district is 3.05 square kilometres (1.18 sq mi)[2] As of the 2010 Census, the total population of the district was 57,133.[3]",
    "The Morgiana Hurdle is a Grade 1 National Hunt hurdle race in Ireland which is open to horses aged four years or older. It is run at Punchestown over a distance of about 2 miles (2 miles and 40 yards, or 3,255 metres), and it is scheduled to take place each year in November.The event was formerly contested over 2½ miles, and for a period it was classed at Listed level. It was cut by 2 furlongs in 1992, and it was given Grade 2 status in 1994. The race was shortened to its present length in 1995 and was promoted to Grade 1 level in 2006."
]

def create_mem_test_asset(lines):
    # repeats ith line i+1 times
    test_lines = []
    for x in range(len(lines)):
        # print([lines[x]] * (x+1))
        test_lines.append([lines[x]] * (x+1))
    return [x for y in test_lines for x in y]

package_and_upload(unique_lines, 'val.json', 's3://anuragik-dev/across_similar_test/val/')


package_and_upload(create_mem_test_asset(unique_lines), 'train1.json', 's3://anuragik-dev/across_similar_test/train/')
package_and_upload(unique_lines, 'train2.json', 's3://anuragik-dev/across_similar_test/train/')



removing val.json
INFO:s3_accessor:Uploading val.json to s3://anuragik-dev/across_similar_test/val/val.json
removing train1.json
INFO:s3_accessor:Uploading train1.json to s3://anuragik-dev/across_similar_test/train/train1.json
removing train2.json
INFO:s3_accessor:Uploading train2.json to s3://anuragik-dev/across_similar_test/train/train2.json


In [9]:
# expected output should only return the 4th line since it is the only thing that is repeated 5 times.
# or we should have a line wise count 1 -> 2, 2-> 3, ...5 -> 6

# Outcome - has the right answers but also some other smallers strings, that maybe I can get rid of using thresholding?

l = list(range(10))
print(l)
print(l[1:20])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[1, 2, 3, 4, 5, 6, 7, 8, 9]
